In [0]:
import shutil
import numpy as np
import pandas as pd

Now we fetch our tokens data

In [2]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
filename = "All_Entries.json"
shutil.copy2('/content/gdrive/My Drive/'+filename,'.')

All_Entries = pd.read_json(filename).reset_index(drop=True)
All_Entries.begin = All_Entries.begin.astype(int)
All_Entries.end = All_Entries.end.astype(int)
All_Entries.delete = All_Entries.delete.astype(bool)
All_Entries.substr_words = All_Entries.substr_words.astype(int)

In [4]:
import nltk

nltk.download("punkt")
nltk.download("perluniprops")

from nltk.tokenize import TweetTokenizer
tknzr = TweetTokenizer()

from nltk.tokenize.moses import MosesDetokenizer
detokenizer = MosesDetokenizer()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package perluniprops to /root/nltk_data...
[nltk_data]   Unzipping misc/perluniprops.zip.


In [5]:
import shutil

drive_path = "Clean REALEC dumps/" #@param {type:"string"}
filename = "realec_110319_2315.tar.gz" #@param {type:"string"}


shutil.copy2('/content/gdrive/My Drive/'+drive_path+filename,'.')

'./realec_110319_2315.tar.gz'

In [0]:
import tarfile

tar = tarfile.open(filename)
tar.extractall()

Once again we'll just load all the content into a huge `dict` for hell sake:

In [7]:
%%time

Texts = list(set([x[:-4]+'.txt' for x in list(All_Entries["path"])]))
Text_Dict = {path: open(path, 'r', encoding='utf-8-sig').read() for path in Texts}

CPU times: user 191 ms, sys: 82.1 ms, total: 273 ms
Wall time: 281 ms


In [8]:
print(len(Text_Dict))

5554


# Creating a file-wise factsheet

Now let's rearrange this data into a DataFrame with info on number of errors in different files

## Full version

In [61]:
%%time

Tags = ["Correction", "Punctuation", "Spelling", "Capitalisation", "Grammar", "Determiners", "Articles", "Quantifiers", "Verbs", "Tense", "Tense_choice", "Tense_form", "Voice", "Modals", "Verb_pattern", "Intransitive", "Transitive", "Reflexive_verb", "Presentation", "Ambitransitive", "Two_in_a_row", "Verb_Inf", "Verb_Gerund", "Verb_Inf_Gerund", "Verb_Bare_Inf", "Verb_object_bare", "Restoration_alter", "Verb_part", "Get_part", "Complex_obj", "Verbal_idiom", "Prepositional_verb", "Dative", "Followed_by_a_clause", "that_clause", "if_whether_clause", "that_subj_clause", "it_conj_clause", "Participial_constr", "Infinitive_constr", "Gerund_phrase", "Nouns", "Countable_uncountable", "Prepositional_noun", "Possessive", "Noun_attribute", "Noun_inf", "Noun_number", "Prepositions", "Conjunctions", "Adjectives", "Comparative_adj", "Superlative_adj", "Prepositional_adjective", "Adj_as_collective", "Adverbs", "Comparative_adv", "Superlative_adv", "Prepositional_adv", "Numerals", "Pronouns", "Agreement_errors", "Word_order", "Standard", "Emphatic", "Cleft", "Interrogative", "Abs_comp_clause", "Exclamation", "Title_structure", "Note_structure", "Conditionals", "Attributes", "Relative_clause", "Defining", "Non_defining", "Coordinate", "Attr_participial", "Lack_par_constr", "Negation", "Comparative_constr", "Numerical", "Confusion_of_structures", "Vocabulary", "Word_choice", "lex_item_choice", "Often_confused", "lex_part_choice", "Absence_comp_colloc", "Redundant", "Derivation", "Formational_affixes", "Suffix", "Prefix", "Category_confusion", "Compound_word", "Discourse", "Ref_device", "Coherence", "Linking_device", "Inappropriate_register", "Absence_comp_sent", "Redundant_comp", "Absence_explanation", "delete"]

Files_Errors = {
  'path': Texts
}

for error in Tags:
  if error == 'delete':
    Files_Errors[error] = [np.sum((All_Entries['path'] == path[:-4]+'.ann') & (All_Entries['delete'] == True)) for path in Texts]
  else:
    Files_Errors[error] = [np.sum((All_Entries['path'] == path[:-4]+'.ann') & (All_Entries['type'] == error)) for path in Texts]

Files_Errors["Words"] = [len(tknzr.tokenize(Text_Dict[path])) for path in Texts]

Files_Errors = pd.DataFrame(Files_Errors)

CPU times: user 2h 24min 37s, sys: 1.72 s, total: 2h 24min 39s
Wall time: 2h 25min


In [62]:
Files_Errors.sample(7)

,path,Correction,Punctuation,Spelling,Capitalisation,Grammar,Determiners,Articles,Quantifiers,Verbs,...,Discourse,Ref_device,Coherence,Linking_device,Inappropriate_register,Absence_comp_sent,Redundant_comp,Absence_explanation,delete,Words
83,./data/old IELTS/IELTS2016/TSha_7_1.txt,0,0,6,3,0,0,1,0,0,...,0,0,1,0,0,2,0,0,0,187
5220,./data/exam/exam2014/AMe_17_2.txt,0,7,8,0,0,0,2,0,0,...,0,1,1,0,0,5,4,1,0,246
3786,./data/exam/exam2017/ABl_42_1.txt,0,1,1,1,0,0,4,0,0,...,0,0,0,0,0,0,0,1,0,186
1625,./data/old IELTS/IELTS2015/VKo_15_1.txt,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,182
4988,./data/2012-2014/esl_00322.txt,0,0,0,0,0,0,1,0,0,...,2,0,0,0,0,0,0,0,1,214
4438,./data/2012-2014/esl_01166.txt,0,0,1,0,0,0,14,0,0,...,0,0,0,0,0,0,0,0,1,827
2507,./data/exam/exam2017_2/ABl_18_2.txt,0,4,1,0,0,0,7,0,0,...,0,0,0,0,0,0,3,1,3,302


## Time estimate for code to execute

In [24]:
%%time

test = [np.sum((All_Entries['path'] == Texts[i][:-4]+'.ann') & (All_Entries['type'] == "Spelling")) for i in range(len(Texts)) if i < 556]

CPU times: user 7.34 s, sys: 4.98 ms, total: 7.34 s
Wall time: 7.35 s


In [28]:
print("We expect the full code to run for",str(round((7.35*len(Tags)*(len(Texts)/555))/3600, 2)),"hours. So we'll stick to selected types of errors for now and then figure out the full summary.")

We expect the full code to run for 2.15 hours. So we'll stick to selected types of errors for now and then figure out the full summary.


## Short version

In [46]:
%%time

Conventional = ["Generic correction", "Punctuation", "Spelling", "Capitalisation", "Grammar", "Determiners", "Articles", "Quantifiers", "Verbs", "Tense", "Choice of tense", "Tense form", "Voice", "Modals", "Verb pattern", "Intransitive verb", "Transitive verb", "Reflexive verb", "Verb with as", "Ambitransitive verb", "Two verbal forms in the predicate", "Verb + Infinitive", "Verb + Gerund", "Verb + Infinitive OR Gerund", "Verb + Bare Infinitive", "Verb + Object/Addressee + Bare Infinitive", "Infinitive Restoration Alternation", "Verb + Participle", "Get + participle", "Complex-object verb", "Verbal idiom", "Prepositional or phrasal verb", "Dative verb with alternation", "Verb followed by a clause", "Verb + that/WH + Clause", "Verb + if/whether + clause", "Verb + that + Subjunctive clause", "Verb + it + Conj + Clause", "Participial construction", "Infinitive construction", "Gerund phrase", "Nouns", "Countable/uncountable", "Prepositional noun", "Possessive form of a noun", "Noun as an attribute", "Noun + Infinitive", "Noun number", "Prepositions", "Conjunctions", "Adjectives", "Comparative degree of adjectives", "Superlative degree of adjectives", "Prepositional adjective", "Adjective as a collective noun", "Adverbs", "Comparative degree of adverbs", "Superlative degree of adverbs", "Prepositional adverb", "Numerals", "Pronouns", "Agreement", "Word order", "Standard word order", "Emphatic shift", "Cleft sentence", "Interrogative word order", "Incomplete sentence", "Exclamation", "Title structure", "Note structure", "Conditionals", "Attributes", "Relative clause", "Defining relative clause", "Non-defining relative clause", "Coordinate relative clause", "Attributive participial construction", "Parallel constructions", "Negation", "Comparative construction", "Numerical comparison", "Confusion of structures", "Vocabulary", "Word choice", "Choice of lexical item", "Words often confused", "Choice of a part of lexical item", "Absence of certain components of a collocation", "Redundant word(s)", "Word formation", "Derivational affixes", "Formational suffix", "Formational prefix", "Confusion of categories", "Compound word", "Discourse", "Referential device", "Coherence", "Linking device", "Inappropriate register", "Absence of a component in clause or sentence", "Redundant component in clause or sentence", "Absence of necessary explanation or detail", "Deletion"]
Tags = ["Correction", "Punctuation", "Spelling", "Capitalisation", "Grammar", "Determiners", "Articles", "Quantifiers", "Verbs", "Tense", "Tense_choice", "Tense_form", "Voice", "Modals", "Verb_pattern", "Intransitive", "Transitive", "Reflexive_verb", "Presentation", "Ambitransitive", "Two_in_a_row", "Verb_Inf", "Verb_Gerund", "Verb_Inf_Gerund", "Verb_Bare_Inf", "Verb_object_bare", "Restoration_alter", "Verb_part", "Get_part", "Complex_obj", "Verbal_idiom", "Prepositional_verb", "Dative", "Followed_by_a_clause", "that_clause", "if_whether_clause", "that_subj_clause", "it_conj_clause", "Participial_constr", "Infinitive_constr", "Gerund_phrase", "Nouns", "Countable_uncountable", "Prepositional_noun", "Possessive", "Noun_attribute", "Noun_inf", "Noun_number", "Prepositions", "Conjunctions", "Adjectives", "Comparative_adj", "Superlative_adj", "Prepositional_adjective", "Adj_as_collective", "Adverbs", "Comparative_adv", "Superlative_adv", "Prepositional_adv", "Numerals", "Pronouns", "Agreement_errors", "Word_order", "Standard", "Emphatic", "Cleft", "Interrogative", "Abs_comp_clause", "Exclamation", "Title_structure", "Note_structure", "Conditionals", "Attributes", "Relative_clause", "Defining", "Non_defining", "Coordinate", "Attr_participial", "Lack_par_constr", "Negation", "Comparative_constr", "Numerical", "Confusion_of_structures", "Vocabulary", "Word_choice", "lex_item_choice", "Often_confused", "lex_part_choice", "Absence_comp_colloc", "Redundant", "Derivation", "Formational_affixes", "Suffix", "Prefix", "Category_confusion", "Compound_word", "Discourse", "Ref_device", "Coherence", "Linking_device", "Inappropriate_register", "Absence_comp_sent", "Redundant_comp", "Absence_explanation", "delete"]
translate_dict = {e[0]: e[1] for e in zip(Conventional, Tags)}

Selected_Tags = ["Spelling", "Choice of lexical item", "Deletion", "Prepositions", "Agreement", "Noun number", "Confusion of categories", "Referential device", "Capitalisation", "Words often confused"]
Selected_Tags = [translate_dict[error] for error in Selected_Tags]

Files_Selected_Errors = {
  'path': Texts
}

for error in Selected_Tags:
  if error == 'delete':
    Files_Selected_Errors[error] = [np.sum((All_Entries['path'] == path[:-4]+'.ann') & (All_Entries['delete'] == True)) for path in Texts]
  else:
    Files_Selected_Errors[error] = [np.sum((All_Entries['path'] == path[:-4]+'.ann') & (All_Entries['type'] == error)) for path in Texts]

Files_Selected_Errors["Words"] = [len(tknzr.tokenize(Text_Dict[path])) for path in Texts]

Files_Selected_Errors = pd.DataFrame(Files_Selected_Errors)

CPU times: user 11min 15s, sys: 99 ms, total: 11min 15s
Wall time: 11min 15s


In [60]:
Files_Selected_Errors.sample(7)

,path,Spelling,lex_item_choice,delete,Prepositions,Agreement_errors,Noun_number,Category_confusion,Ref_device,Capitalisation,Often_confused,Words
1352,./data/exam/exam2016/NMa_22_1.txt,0,0,0,1,0,0,0,0,0,0,148
3002,./data/exam/exam2017/OBy_90_2.txt,1,4,3,0,0,2,0,0,0,0,299
5334,./data/Exam_practice/OV201617/151-152/st_35_3.txt,3,2,0,5,0,0,0,0,0,0,207
188,./data/Exam_practice/OV_2_year/essays_fr/st_42...,1,2,1,2,0,0,0,2,0,0,178
2783,./data/exam/exam2014/2012-2014_4/esl_00858.txt,1,4,2,1,1,0,0,0,0,0,449
4264,./data/exam/exam2014/EPa_35_1.txt,2,1,1,2,0,0,0,0,0,0,211
1399,./data/exam/exam2016/OR_53_1.txt,0,2,1,0,1,1,0,0,0,0,191


In [47]:
total = Files_Selected_Errors.sum(numeric_only=True)
total["Words"]

1513087

Let's count the overall percentage of each error type:

In [48]:
percentage = {el: total[el]/total['Words'] for el in total.keys()}

percentage

{'Agreement_errors': 0.0028464985820379133,
 'Capitalisation': 0.0007679664156786754,
 'Category_confusion': 0.0015855003710956476,
 'Noun_number': 0.0020732449621204863,
 'Often_confused': 0.000760696509850392,
 'Prepositions': 0.003476997687509046,
 'Ref_device': 0.000993994396885308,
 'Spelling': 0.009436337765111985,
 'Words': 1.0,
 'delete': 0.004556909153274068,
 'lex_item_choice': 0.007345248488685713}

As our previous research has shown that datasets with 1:15 erroneous to non-erroneous ratio yield the best results, let's try to scale all our error types to this ratio. The ratio for *Choice of lexical item* should remaing unchanged (around `0.6666667`):

In [49]:
augmented_ratio = {el: percentage[el]*((1/15)/percentage['lex_item_choice']) for el in percentage.keys()}

augmented_ratio

{'Agreement_errors': 0.02583528282646512,
 'Capitalisation': 0.006970187751184692,
 'Category_confusion': 0.014390258532781476,
 'Noun_number': 0.01881710755203647,
 'Often_confused': 0.006904204906724252,
 'Prepositions': 0.03155779497330694,
 'Ref_device': 0.009021654369863836,
 'Spelling': 0.08564573210965148,
 'Words': 9.076162197828564,
 'delete': 0.04135924659588507,
 'lex_item_choice': 0.06666666666666667}

In [50]:
Summary = {
    "Percentage": percentage,
    "Augmented ratio": augmented_ratio
}

Summary = pd.DataFrame(Summary)

Summary = Summary.sort_values('Percentage', ascending=0)

Summary

,Percentage,Augmented ratio
Words,1.000000,9.076162
Spelling,0.009436,0.085646
lex_item_choice,0.007345,0.066667
delete,0.004557,0.041359
Prepositions,0.003477,0.031558
Agreement_errors,0.002846,0.025835
Noun_number,0.002073,0.018817
Category_confusion,0.001586,0.014390
Ref_device,0.000994,0.009022
Capitalisation,0.000768,0.006970


In [51]:
Files_Selected_Errors.sample(7)

,path,Spelling,lex_item_choice,delete,Prepositions,Agreement_errors,Noun_number,Category_confusion,Ref_device,Capitalisation,Often_confused,Words
833,./data/Academic_Writing/2_exercise/student7.txt,0,0,0,0,1,0,0,0,0,0,364
3642,./data/exam/exam2016/JSl_122_2.txt,0,0,2,0,0,0,0,0,0,0,280
2253,./data/old IELTS/IELTS2016/EKu_12_1.txt,0,2,0,0,1,0,2,0,0,0,218
4911,./data/exam/exam2017/OBy_138_2.txt,2,0,0,0,0,0,0,0,0,0,297
1426,./data/exam/exam2017/VSa_9_1.txt,1,2,3,1,0,0,1,0,0,0,132
4692,./data/exam/exam2014/2012-2014_4/esl_00904.txt,4,2,1,1,1,0,0,4,0,0,375
2698,./data/old IELTS/IELTS2016/DZu_75_2.txt,4,5,0,0,0,0,0,0,0,0,319


## Uploading the results

Wow. That was epic and took a looong time. We'd better save it into a json file to accelerate future load-ups.

In [63]:
%%time

jsonname = "Files_Errors.json" #@param {type:"string"}
with open(jsonname, 'w', encoding="utf-8") as outie:
    exec("outie.write("+jsonname[:-5]+".to_json())")

CPU times: user 119 ms, sys: 23 ms, total: 142 ms
Wall time: 148 ms


Whew, great! Let's upload this to Google Drive right now. 

In [0]:
# Install the PyDrive wrapper & import libraries.
# This only needs to be done once in a notebook.
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once in a notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [66]:
# Create & upload a file.
uploaded = drive.CreateFile({'title': jsonname})
uploaded.SetContentFile(jsonname)
uploaded.Upload()
print('Uploaded file with ID {}'.format(uploaded.get('id')))

Uploaded file with ID 1K630IK8-sZ4DfjAUPqCNbJ_PU7Qc9Qdz
